In [1]:
import numpy as np
import pandas as pd
import re
from nltk.stem import PorterStemmer
import math
import collections

In [2]:
## Parsing
# Documents
with open('TIME.ALL', 'r') as f:
    text = f.read()
result = re.findall(r"\*TEXT\s+(\d{3})\s+(\d{2}/\d{2}(/|\s)\d{2})\s+PAGE\s+(\d{3})\n\n(.+?)(?=\*TEXT|$)", text, re.DOTALL)
docDB = {match[0]: {'id': match[0], 'date': match[1], 'page': int(match[3]), 'text': match[4]} for match in result}
docOnlyDB = {match[0]: match[4] for match in result}
docDF = pd.DataFrame(docDB).T

# Queries
with open('TIME.QUE', 'r') as f:
    text = f.read()
queryDB = re.findall(r'FIND\s+\d+\s+(.+?)(?=\n\n\*FIND\s+\d+\s+|$)', text, re.DOTALL)

# Stopwords
with open('TIME.STP','r') as f:
    text = f.read()
swDB = re.findall(r"^[A-Z]+$", text, re.MULTILINE)
swDB = set([word.lower() for word in swDB])

# Relevant docs
with open('TIME.REL', 'r') as f:
    text = f.read()
    lines = text.split("\n")
rdDB = {}
for line in lines:
    numbers = re.findall(r"\d+", line)
    if numbers:
        key = numbers[0]
        values = [int(n) for n in numbers[1:]]
        rdDB[key] = values

In [3]:
# Tokenize a document
def tokenizeDocument(documentText):
    return documentText.split()

# Normalize and Stop the token stream of a document
def normalizeAndStopTokenStream(tokenStream):
    return [token.lower() for token in tokenStream if token.lower().isalnum() and token.lower() not in swDB]

# Stem and the normalized token stream of a document
def stemNormalizedTokenStream(normalizedTokenStream):
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(token) for token in normalizedTokenStream])

def processDocument(documentText):
    return stemNormalizedTokenStream(normalizeAndStopTokenStream(tokenizeDocument(documentText)))

In [4]:
processedDocDB = {docID: processDocument(text) for docID, text in docOnlyDB.items()}
processedDocDB

{'017': 'alli nassau decemb propos help nato develop nuclear strike forc europ attempt devis plan studi nassau accord presid kennedi prime minist european saw emerg outlin nuclear nato want support sprang crisi cancel skybolt offer suppli britain franc polari dec alli leader unreservedli welcom polari offer harold keep separ nuclear deterr britain save neck prime minist beam britain weapon gener term briton sure govern shoulder none develop cost pour skybolt spend billion fleet submarin british abl build prove nuclear fleet bomber forc presum obsolet tori backbench loudli skeptic call type nassau stipul polari submarin except suprem nation interest commit truli multilater nato forc mean britain eventu strike forc decid nation interest justifi withdraw submarin particularli nation interest conflict polici inclus offer deliber ploy jack kennedi end downgrad prize special relationship cartoonist farther supermac jump de gaull adenau edith help develop forc de quick crow vaunt tie humili b

In [5]:
allTerms = sorted(list(set([token for doc in processedDocDB.values() for token in doc.split()])))
allTerms

['00',
 '1',
 '10',
 '100',
 '100th',
 '101',
 '1013',
 '103',
 '104',
 '105',
 '108',
 '109',
 '10th',
 '11',
 '113',
 '115',
 '116',
 '117',
 '1170',
 '119',
 '12',
 '120',
 '125',
 '126',
 '128',
 '12f',
 '12in',
 '13',
 '130',
 '131',
 '133',
 '136',
 '137',
 '138',
 '139',
 '13th',
 '14',
 '140',
 '144',
 '1498',
 '14th',
 '15',
 '150',
 '1512',
 '155',
 '156',
 '158',
 '159',
 '15f',
 '15member',
 '15q6',
 '15th',
 '16',
 '160',
 '162',
 '1630',
 '165',
 '166',
 '16th',
 '17',
 '170',
 '172',
 '1735',
 '1745',
 '175',
 '1750',
 '1787',
 '17th',
 '18',
 '180',
 '1802',
 '1805',
 '182',
 '1820',
 '1825',
 '1829',
 '1830',
 '1841',
 '1848',
 '185',
 '1862',
 '1864',
 '1868',
 '1872',
 '1878',
 '1887',
 '1890',
 '1894',
 '1899',
 '18th',
 '19',
 '190',
 '1902',
 '1904',
 '1912',
 '1916',
 '1917',
 '1919',
 '1920',
 '1921',
 '1922',
 '1924',
 '1925',
 '1926',
 '1928',
 '1929',
 '1930',
 '1931',
 '1932',
 '1933',
 '1936',
 '1937',
 '1938',
 '1939',
 '194',
 '1940',
 '1941',
 '1942',
 '

In [6]:
# No. of documents a term appears in the DB
def df(term, documentDB):
    return len([1 for document in documentDB if term in document])

# Inverse document frequency or informativeness of a term
def idf(term, documentDB):
    return math.log((len(documentDB) + 1) / (df(term, documentDB) + 0.5))

# Precalculating and making a cache
idfMap = {term:idf(term, processedDocDB.values()) for term in allTerms}
def getIDF(term):
    return idfMap[term]

# Calculate weights for a document
def calculateDocumentWeight(document):
    counter = collections.Counter(document.split())
    return [counter.get(term, 0) * getIDF(term) for term in allTerms]

In [7]:
tdMatrixDF = pd.DataFrame([calculateDocumentWeight(processedDoc) for processedDoc in processedDocDB.values()], columns=allTerms, index=processedDocDB.keys())

In [8]:
processedQueryDB = [processDocument(query) for query in queryDB]
processedQueryDB

['kennedi administr pressur ngo dinh diem stop suppress buddhist',
 'effort ambassador henri cabot lodg viet presid diem chang polici polit repress',
 'number troop unit state station south viet nam compar number troop station west germani',
 'polici new regim south viet nam overthrew presid diem',
 'person involv viet nam coup',
 'ceremoni suicid commit buddhist monk south viet nam seek gain act',
 'reject princ norodom asian neutralist further aid nation',
 'team survey public opinion north borneo sarawak join feder malaysia',
 'opposit indonesia malaysia',
 'grow controversi southeast asia propos creation feder malaysia',
 'arrang indonesia administr west unit nation administr',
 'controversi indonesia malaya propos feder unit territori',
 'precari truce lao britain 14 nation agre truce geneva',
 'form part name discuss intern relat east',
 'elect park chung hee presid south korea',
 'effort intern control commiss tri stop fight flare lao',
 'withdraw sultan brunei propos feder mala

In [9]:
def cosineSimiliary(queryWeights, tdMatrix):
    return np.dot(tdMatrix, queryWeights) / (np.linalg.norm(queryWeights) * np.linalg.norm(tdMatrix, axis=1))

def findTopNRelevantDocsWithCosineSimilarity(query, tdMatrixDF, N):
    # Calculate the cosine similarity
    cosineSimilarities = cosineSimiliary(calculateDocumentWeight(query), tdMatrixDF.values)

    # Sort in descending order of cosine similarity
    df = pd.DataFrame({'docID': tdMatrixDF.index, 'cosineSimilarity': cosineSimilarities})
    sorted_df = df.sort_values(by='cosineSimilarity', ascending=False)

    # Return the top 10 relevant documents from the sorted dataframe
    return sorted_df['docID'].values[:N].tolist()

In [10]:
cosineSimiliaryResults = {str(idx + 1):findTopNRelevantDocsWithCosineSimilarity(processedQuery, tdMatrixDF, 10) for idx, processedQuery in enumerate(processedQueryDB)}
cosineSimiliaryResults

{'1': ['396', '320', '414', '418', '498', '334', '390', '434', '464', '363'],
 '2': ['498', '434', '418', '518', '464', '390', '414', '228', '269', '396'],
 '3': ['060', '269', '405', '029', '464', '434', '470', '319', '021', '384'],
 '4': ['434', '498', '269', '464', '228', '518', '418', '390', '396', '470'],
 '5': ['533', '269', '464', '390', '518', '434', '559', '470', '522', '202'],
 '6': ['396', '414', '418', '334', '320', '390', '415', '363', '434', '533'],
 '7': ['149', '122', '544', '300', '519', '361', '226', '201', '227', '553'],
 '8': ['479', '443', '094', '203', '389', '270', '204', '335', '303', '265'],
 '9': ['479', '335', '303', '204', '094', '443', '256', '203', '299', '389'],
 '10': ['335', '479', '094', '203', '204', '389', '303', '321', '443', '192'],
 '11': ['256', '183', '196', '335', '299', '043', '063', '317', '048', '094'],
 '12': ['203', '335', '094', '256', '204', '479', '303', '192', '389', '321'],
 '13': ['243', '227', '300', '226', '122', '113', '361', '446

**BIM**

Considering only one query here.

In [11]:
def trk(term, relevantDocsDB, nonRelevantDocsDB):
    rk = df(term, relevantDocsDB.values())
    nrk = df(term, nonRelevantDocsDB.values())
    Opk = (rk+0.5) / (len(relevantDocsDB.values())-rk+0.5)
    Oqk = (nrk+0.5) / (len(nonRelevantDocsDB.values())-nrk+0.5)
    return np.log10(Opk/Oqk)

# Phase 1 of BIM assumes all the documents to be non-relevant when computing the term score (t<sub>rk</sub>)
trkMap = {term:trk(term, {}, processedDocDB) for term in allTerms}

def getWeightVector(trk, document):
    tokenStream = set(document.split())
    return [trk[term] if term in tokenStream else 0 for term in allTerms]

def generateTDMDataFrame(trk, processedDocDB):
    return pd.DataFrame([getWeightVector(trk, processedDoc) for processedDoc in processedDocDB.values()], columns=allTerms, index=processedDocDB.keys())

In [12]:
def simpleSimiliaryForBIM(query, tdMatrix):
    queryWeights = getWeightVector(trkMap, query)
    return np.dot(tdMatrix, queryWeights)

def findTopNRelevantDocsWithBIM(query, tdMatrix, N):
    similarities = simpleSimiliaryForBIM(query, tdMatrix.values)
    
    df = pd.DataFrame({'docID': tdMatrix.index, 'similarity': similarities})
    sorted_df = df.sort_values(by='similarity', ascending=False)
    return sorted_df['docID'].values[:N].tolist()

In [13]:
def BIMPhase1(trk, queryDB, documentDB, N):
    tdMatrix = generateTDMDataFrame(trk, documentDB)
    return {str(queryID + 1):findTopNRelevantDocsWithBIM(query, tdMatrix, N) for queryID, query in enumerate(queryDB)}

In [14]:
BIMPhase1Top10 = BIMPhase1(trkMap, processedQueryDB, processedDocDB, 10)
BIMPhase1Top10

{'1': ['320', '464', '480', '163', '363', '533', '390', '334', '414', '396'],
 '2': ['418', '434', '464', '414', '518', '498', '390', '112', '425', '128'],
 '3': ['370', '405', '449', '319', '556', '464', '196', '200', '396', '029'],
 '4': ['357', '087', '318', '418', '414', '434', '518', '269', '480', '498'],
 '5': ['533', '418', '163', '252', '487', '449', '227', '396', '368', '152'],
 '6': ['390', '415', '480', '334', '498', '418', '414', '396', '320', '533'],
 '7': ['544', '081', '519', '546', '561', '204', '243', '485', '054', '227'],
 '8': ['443', '479', '204', '335', '203', '270', '094', '389', '368', '265'],
 '9': ['204', '443', '303', '479', '094', '335', '203', '256', '299', '389'],
 '10': ['094', '204', '464', '203', '335', '396', '187', '123', '449', '443'],
 '11': ['204', '071', '048', '256', '479', '203', '443', '303', '335', '094'],
 '12': ['204', '094', '203', '335', '303', '479', '464', '256', '443', '243'],
 '13': ['227', '361', '449', '300', '243', '545', '561', '226

In [15]:
def recomputeTrk(relDocs, documentDB):
    relevantDocs, nonRelevantDocs = {}, {}
    relDocSet = set(relDocs)
    for docID, text in documentDB.items():
        if docID in relDocSet:
            relevantDocs[docID] = text
        else:
            nonRelevantDocs[docID] = text
    return {term:trk(term, relevantDocs, nonRelevantDocs) for term in allTerms}

In [16]:
def BIMPhase2(BIMPhase1Result, queryDB, documentDB):
    BIMPhase2TopN = {}
    for queryID, query in enumerate(queryDB):
        queryRelDocsFromPhase1 = BIMPhase1Result[str(queryID+1)]
        trkAdjusted = recomputeTrk(queryRelDocsFromPhase1, documentDB)
        tdMatrixAdjusted = generateTDMDataFrame(trkAdjusted, documentDB)
        BIMPhase2TopN[str(queryID+1)] = findTopNRelevantDocsWithBIM(query, tdMatrixAdjusted, 10)
    return BIMPhase2TopN

In [17]:
BIMPhase2Top10 = BIMPhase2(BIMPhase1Top10, processedQueryDB[:5], processedDocDB)
BIMPhase2Top10

{'1': ['320', '480', '464', '390', '533', '363', '334', '498', '418', '434'],
 '2': ['418', '434', '464', '414', '518', '498', '390', '425', '112', '516'],
 '3': ['370', '449', '319', '556', '464', '200', '029', '405', '396', '163'],
 '4': ['087', '357', '318', '414', '434', '418', '390', '320', '518', '519'],
 '5': ['533', '163', '418', '487', '252', '449', '369', '368', '152', '396']}

In [18]:
# First 5 query results only because BIM takes 7 seconds per query 💀
givenRelDocs = dict(list(rdDB.items())[:5])

cosSimRelDocs = dict(list(cosineSimiliaryResults.items())[:5])
BIMRelDocs = BIMPhase2Top10

In [19]:
def metrics(predicted, actual):
    intersect = set(predicted) & set(actual)
    precision = len(intersect) / len(set(predicted))
    recall = len(intersect) / len(set(actual))
    f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    return precision, recall, f1

cosSim_metrics = {q: metrics(cosSimRelDocs[q], givenRelDocs[q]) for q in givenRelDocs}
BIM_metrics = {q: metrics(BIMRelDocs[q], givenRelDocs[q]) for q in givenRelDocs}

metricDF = pd.concat([pd.DataFrame(m).T.assign(Algorithm=a) for m, a in [(cosSim_metrics, 'Cosine Similarity'), (BIM_metrics, 'BIM')]])
metricDF.columns = ['Precision', 'Recall', 'F1-Measure', 'Algorithm']
metricDF

,Precision,Recall,F1-Measure,Algorithm
1,0.0,0.0,0.0,Cosine Similarity
2,0.0,0.0,0.0,Cosine Similarity
3,0.0,0.0,0.0,Cosine Similarity
4,0.0,0.0,0.0,Cosine Similarity
5,0.0,0.0,0.0,Cosine Similarity
1,0.0,0.0,0.0,BIM
2,0.0,0.0,0.0,BIM
3,0.0,0.0,0.0,BIM
4,0.0,0.0,0.0,BIM
5,0.0,0.0,0.0,BIM


In conclusion, both algorithms _**suck**_.

## Rocchio Relevance Feedback

**What we have**: original queries, tdMatrix and cosine similarity results (for e.g.)

**What we'll do**: recompute query weights with rocchio's forumla

In [20]:
def findBetterTopNWithRocchio(queryID, tdMatrix, cosine_similarity_results, N=10, alpha=1, beta=0.75, gamma=0.25):
    query = processedQueryDB[queryID]
    queryWeightVector = calculateDocumentWeight(query)

    all_docs = set(processedDocDB.keys())
    relevant_docs = cosine_similarity_results[str(queryID+1)]
    non_relevant_docs = list(all_docs - set(relevant_docs))

    relevant_centroid = np.mean(tdMatrix.loc[relevant_docs], axis=0)
    non_relevant_centroid = np.mean(tdMatrix.loc[non_relevant_docs], axis=0)

    recomputedQueryWeights = alpha * queryWeightVector + beta * relevant_centroid - gamma * non_relevant_centroid
    cosineSimilarities = cosineSimiliary(recomputedQueryWeights, tdMatrixDF.values)

    df = pd.DataFrame({'docID': tdMatrixDF.index, 'cosineSimilarity': cosineSimilarities})
    sorted_df = df.sort_values(by='cosineSimilarity', ascending=False)

    return sorted_df['docID'].values[:N].tolist()

In [21]:
rocchioRFResults = {queryID:findBetterTopNWithRocchio(queryID, tdMatrixDF, cosineSimiliaryResults) for queryID, _ in enumerate(processedQueryDB)}
rocchioRFResults

{0: ['418', '396', '320', '434', '414', '334', '390', '498', '464', '363'],
 1: ['418', '434', '396', '498', '390', '464', '414', '320', '269', '334'],
 2: ['464', '269', '434', '029', '470', '418', '498', '545', '518', '390'],
 3: ['418', '434', '396', '498', '464', '390', '320', '269', '414', '334'],
 4: ['464', '434', '269', '390', '518', '533', '498', '418', '470', '396'],
 5: ['418', '396', '320', '414', '334', '390', '434', '363', '415', '498'],
 6: ['226', '122', '361', '227', '300', '243', '201', '149', '519', '553'],
 7: ['204', '479', '443', '094', '203', '389', '303', '335', '270', '299'],
 8: ['204', '479', '443', '094', '303', '389', '203', '335', '299', '256'],
 9: ['204', '479', '094', '389', '203', '303', '443', '335', '192', '321'],
 10: ['196', '183', '317', '094', '299', '063', '335', '048', '256', '043'],
 11: ['204', '479', '094', '389', '203', '303', '335', '443', '192', '256'],
 12: ['243', '226', '361', '122', '227', '300', '280', '113', '201', '446'],
 13: ['35

The above is the recomputed relevant docs using Rocchio Relevance Feedback for all the 83 queries in the document.